In [1]:

#Install required libraries
import pandas as pd
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.query import Gen3Query

In [2]:
### download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
# import os
# os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py")


In [3]:
from stress_testing.expansion import Gen3Expansion

In [4]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
env_name = 'my_env_name'
api = f'https://{env_name}.planx-pla.net/' #'https://qa-midrc.planx-pla.net/'
cred = f'{env_name}_api_key.json' #'midrc_api_key.json'



In [5]:

auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()

Getting all project_ids you have access to in https://georget.planx-pla.net/
['DEV-JianProjectDev', 'DEV-project1', 'DEV-test', 'jnkns-jenkins', 'jnkns-jenkins3', 'prog1-proj1']


['DEV-JianProjectDev',
 'DEV-project1',
 'DEV-test',
 'jnkns-jenkins',
 'jnkns-jenkins3',
 'prog1-proj1']

In [6]:
project_id = 'CGCI-load_test1' if env_name == 'piotr' else 'DEV-test'
img_stud = exp.get_node_tsvs(projects = project_id, node='demographic', overwrite=True)


Output written to file: node_tsvs/demographic_tsvs/DEV-test_demographic.tsv
node_tsvs/demographic_tsvs/DEV-test_demographic.tsv has 22 records.
length of all dfs: 22
Master node TSV with 22 total records written to master_demographic.tsv.


In [7]:
range_var = 4000
#increasing the volume of the number of records by 10 fold for testing purposes
img_stud_list = [img_stud.copy(deep=True) for i in range(range_var) ]
for i in range(range_var):
    img_stud_list[i]["submitter_id"] = img_stud['submitter_id']+"_"+str(i) #changing the submitter_id to make them unique

img_stud = pd.concat(img_stud_list)

type
submitter_id	
datasets
cases

In [8]:
# Trim down the number of records to 4000 to save time on submission
len(img_stud)
img_stud = img_stud.head(4000)
len(img_stud)

4000

In [9]:
# img_stud = img_stud[['type', 'submitter_id', 'cases.submitter_id']]

# img_stud['datasets.submitter_id'] = 'dataset_backpedalled_usurers'

img_stud.to_csv('test_img_stud.tsv', sep='\t', index=False)



In [10]:
import time
start = time.time()
sub_test = sub.submit_file(project_id = project_id, filename = 'test_img_stud.tsv', chunk_size=1000)
end = time.time()
time_taken = end - start



Submitting test_img_stud.tsv with 4000 records.
Chunk 1 (chunk size: 1000, submitted: 0 of 4000)
	 Succeeded: 1000 entities.
Chunk 2 (chunk size: 1000, submitted: 1000 of 4000)
	 Succeeded: 1000 entities.
Chunk 3 (chunk size: 1000, submitted: 2000 of 4000)
	 Succeeded: 1000 entities.
Chunk 4 (chunk size: 1000, submitted: 3000 of 4000)
	 Succeeded: 1000 entities.
Chunk 5 (chunk size: 1000, submitted: 4000 of 4000)
	Chunk Failed (status code 400): 0 entities.
	Invalid records in this chunk: 0
Finished data submission.
Successful records: 4000
Failed invalid records: 0
